# Order-free matching performance

In [1]:
# Generic imports

import glob
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import ast
import json


# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

from tqdm import tqdm

### Read order-free candidates

In [2]:
picos = 'I' # ['P', 'I', 'O', 'S']
match_level = 'doc' # ['doc', 'sent', 'win_5', 'para']

In [ ]:
order_free_dir = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_free/{match_level}/{picos}'
order_free_files = os.listdir(order_free_dir)
print('Files: ', order_free_files)

In [ ]:
order_free_files.remove('.nfs0000000011200a4200000003')

In [ ]:
orf_loaded_files = dict()

for i in tqdm(order_free_files):
    
    filpath = f'{order_free_dir}/{i}'
    print('Loading file...', i)
    with open( filpath, 'r' ) as rf:
        orf_i = json.load(rf)
        orf_loaded_files[i] = orf_i

In [ ]:
def get_orf(orf, partial: bool):
        
    orf_doc_offsets = dict()

    for i, (k, v) in enumerate(orf.items()):

        if i == 0:
            continue

        if 'Inters. (full)' in v and len(v['Inters. (full)']) > 0 and len(v['tokens']) > 1:

            for k_i, v_i in v['Inters. (full)'].items():

                orf_doc_offsets[k_i] = []

                if len( v_i['tokens'] ) > 1:

                    source_tokens = v_i['tokens']
                    source_offsets = v_i['char offs.']
                    orf_doc_offsets[k_i].extend( source_offsets )
                    
        if partial == True:
            
            if 'Inters. (partial)' in v and len(v['Inters. (partial)']) > 0 and len(v['tokens']) > 1:

                for k_i, v_i in v['Inters. (partial)'].items():

                    orf_doc_offsets[k_i] = []

                    if len( v_i['tokens'] ) > 1:

                        source_tokens = v_i['tokens']
                        source_offsets = v_i['char offs.']
                        orf_doc_offsets[k_i].extend( source_offsets )
            
    
    return orf_doc_offsets

In [ ]:
orf_int_syn_offsets_par = get_orf(orf_loaded_files['train_ebm_intervention_syn.json'], partial = True)
orf_int_syn_offsets_full = get_orf(orf_loaded_files['train_ebm_intervention_syn.json'], partial = False)

In [ ]:
orf_int_offsets_par = get_orf(orf_loaded_files['train_ebm_intervention.json'], partial = True)
orf_int_offsets_full = get_orf(orf_loaded_files['train_ebm_intervention.json'], partial = False)

In [ ]:
print(len(orf_int_syn_offsets_par))
print(len(orf_int_syn_offsets_full))

In [ ]:
print(len(orf_int_offsets_par))
print(len(orf_int_offsets_full))

### Merge dictionaries

In [ ]:
def FullMergeDict(D1, D2):
    
    for key, value in D1.items():
        if key in D2:
            if type(value) is dict:
                FullMergeDict(D1[key], D2[key])
            else:
                if type(value) in (int, float, str):
                    D1[key] = [value]
                if type(D2[key]) is list:
                    D1[key].extend(D2[key])
                else:
                    D1[key].append(D2[key])
    for key, value in D2.items():
        if key not in D1:
            D1[key] = value
            
    return D1

In [ ]:
def merge_dics(D1, D2):
    
    for k,v in D2.items():
        if k in D1:
            #merge the values
            D2_val = D2[k]
            D1[k].extend( D2_val )
        else:
            D1[k] = v
    
    return D1

In [ ]:
orf_offsets_par_merged = merge_dics(orf_int_syn_offsets_par, orf_int_offsets_par)

In [ ]:
print( len(orf_offsets_par_merged) )

In [ ]:
print( len(orf_offsets_par_merged) )

In [ ]:
orf_offsets_full_merged = merge_dics(orf_int_syn_offsets_full, orf_int_offsets_full)

In [ ]:
print( len(orf_offsets_full_merged) )

In [ ]:
print( len(orf_offsets_full_merged) )

### Load order-bound matches

In [ ]:
def order_free_matches(x, orf_offsets):
    
    labs_modified = []
    
    for i, (identifier, offs, labs) in enumerate( zip(x.pmid, x.offsets, x.labels) ):
             
        lab_val = [v for k, v in ast.literal_eval(labs).items()] 
        off_val = ast.literal_eval(offs) 
        
        if str(identifier) in orf_offsets: 
            orf_matches =  orf_offsets[ str(identifier) ]
            match_indices = [ off_val.index(m) for m in orf_matches ]
            for i, l in enumerate(lab_val):
                if i in match_indices:
                    lab_val[i] = 1
                    
        labs_modified.append( lab_val )
        
        
    return labs_modified

In [ ]:
order_bound_file = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/direct/{picos}/lf_ds_intervention_syn.tsv'

In [ ]:
data = pd.read_csv(order_bound_file, sep='\t', header=0)

In [ ]:
# Fetch ground truth

# coarse
picos_coarse_ = data[str(picos.lower())]
picos_coarse = [ l for ent_series in picos_coarse_ for l in ast.literal_eval(ent_series) ]

# fine
picos_fine_ = data[ str(picos.lower()) + '_f' ]
picos_fine = [ l for ent_series in picos_fine_ for l in ast.literal_eval(ent_series) ]
picos_fine = ['1' if (n != '1' and n != '0') else str(n) for i, n in enumerate(picos_fine) ]

In [ ]:
# Fetch order-bound predictions

preds = [ v for ent_series in data['labels'] for k, v in ast.literal_eval(ent_series).items() ]
order_bound_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds) ]

In [ ]:
# Fetch order-free predictions (dictionary int_syn)

# ORF with partial
data['orf_labels_partial'] = order_free_matches( data, orf_int_syn_offsets_par )
preds_orf_doc_partial = [ v for ent_series in data['orf_labels_partial'] for v in ent_series ]
order_free_partial_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_doc_partial) ]

# ORF with full
data['orf_labels_full'] = order_free_matches( data, orf_int_syn_offsets_full )
preds_orf_doc_full = [ v for ent_series in data['orf_labels_full'] for v in ent_series ]
order_free_full_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_doc_full) ]

In [ ]:
# Fetch order-free predictions (dictionary expansion int + int_syn)

# ORF with partial
data['orf_labels_merged_partial'] = order_free_matches( data, orf_offsets_par_merged )
preds_orf_merged_partial = [ v for ent_series in data['orf_labels_merged_partial'] for v in ent_series ]
order_free_merged_partial_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_merged_partial) ]

# ORF with full
data['orf_labels_merged_full'] = order_free_matches( data, orf_offsets_full_merged )
preds_orf_merged_full = [ v for ent_series in data['orf_labels_merged_full'] for v in ent_series ]
order_free_merged_full_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_merged_full) ]

### evaluation metrics

##### direct matching

In [ ]:
# Classification report
cr_order_bound_coarse = classification_report( picos_coarse, order_bound_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and order-bound matches')
print( cr_order_bound_coarse )

cr_order_bound_fine = classification_report( picos_fine, order_bound_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and order-bound matches')
print( cr_order_bound_fine )

#### ORF matching - int_syn

In [ ]:
# Classification report
cr_order_free_doc_full_coarse = classification_report( picos_coarse, order_free_partial_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free matches')
print( cr_order_free_doc_full_coarse )

cr_order_free_doc_full_fine = classification_report( picos_fine, order_free_partial_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free matches')
print( cr_order_free_doc_full_fine )

In [ ]:
# Classification report
cr_order_free_merged_full_coarse = classification_report( picos_coarse, order_free_merged_partial_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free matches')
print( cr_order_free_merged_full_coarse )

cr_order_free_merged_full_fine = classification_report( picos_fine, order_free_merged_partial_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free matches')
print( cr_order_free_merged_full_fine )

In [ ]:
# Classification report
cr_order_free_doc_partial_coarse = classification_report( picos_coarse, order_free_full_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_doc_partial_coarse )

cr_order_free_doc_partial_fine = classification_report( picos_fine, order_free_doc_full_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_doc_partial_fine )

In [ ]:
# Classification report
cr_order_free_merged_partial_coarse = classification_report( picos_coarse, order_free_merged_full_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_merged_partial_coarse )

cr_order_free_merged_partial_fine = classification_report( picos_fine, order_free_merged_full_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_merged_partial_fine )